In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix

import joblib


In [2]:
df = pd.read_csv("../datasets/classification.csv",encoding='ISO-8859-1')
df.head()



,Text,Label
0,"I keep imagining the worst possible outcome, e...",Anxiety
1,"My heart races before every phone call, even c...",Anxiety
2,I cant stop overthinking what I said in that ...,Anxiety
3,Every little noise makes me jump lately.,Anxiety
4,I feel like something bad is about to happen a...,Anxiety


In [3]:
df.sample(n=5)

,Text,Label
434,I feel like I dont deserve to be happy.,Post-Traumatic Stress Disorder (PTSD)
207,"I procrastinate until I panic, then work in a ...",ADHD( Attention deficit hyperactivity disorder)
51,"If I dont check the door three times, I panic...",Obsessive-Compulsive Disorder (OCD)
75,"I try not to think about it, but it keeps comi...",Post-Traumatic Stress Disorder (PTSD)
353,I cancel plans if I overeat earlier in the day.,Eating Disorder


In [4]:
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

Shape: (450, 2)
Columns: ['Text', 'Label']


In [5]:
print(df['Label'].value_counts(normalize=True))


Label
Anxiety                                            0.111111
Depression                                         0.111111
ADHD( Attention deficit hyperactivity disorder)    0.111111
Bipolar Disorder                                   0.111111
Autism Spectrum Disorder (ASD)                     0.111111
Obsessive-Compulsive Disorder (OCD)                0.111111
Schizophrenia                                      0.111111
Post-Traumatic Stress Disorder (PTSD)              0.111111
Eating Disorder                                    0.111111
Name: proportion, dtype: float64


In [6]:
# Check missing/nulls
print(df.isnull().sum())




Text     0
Label    0
dtype: int64


In [7]:
#Text Preprocessing


In [8]:
# Lowercase
df['Text'] = df['Text'].str.lower()

# Remove non-ASCII characters (already added)
df['Text'] = df['Text'].str.encode('ascii', 'ignore').str.decode('ascii')

# Remove punctuation
df['Text'] = df['Text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# Remove extra whitespace
df['Text'] = df['Text'].apply(lambda x: re.sub('\s+', ' ', x).strip())



In [9]:
#Augmentation

In [12]:
!pip install transformers sentencepiece
#Here’s code that:

#Loads a T5 model for paraphrasing

#Augments 1–3 paraphrases per sample (configurable)

#Ensures new rows retain the same label




In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm

# Load T5-base paraphrasing model
paraphrase_tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5_paraphraser")
paraphrase_model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5_paraphraser")
paraphrase_model.eval()

# If you have a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
paraphrase_model = paraphrase_model.to(device)

def paraphrase_text(text, num_return_sequences=2, num_beams=5):
    input_text = f"paraphrase: {text} </s>"
    encoding = paraphrase_tokenizer.encode_plus(input_text, padding='max_length', max_length=128, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    outputs = paraphrase_model.generate(
        input_ids=input_ids,
        attention_mask=attention_masks,
        max_length=128,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
        temperature=1.5
    )

    paraphrases = [paraphrase_tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                   for output in outputs]

    return paraphrases


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

C:\Users\Asus\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asus\.cache\huggingface\hub\models--ramsrigouthamg--t5_paraphraser. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [14]:
augmented_texts = []
augmented_labels = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    original_text = row['Text']
    label = row['Label']

    try:
        paraphrases = paraphrase_text(original_text, num_return_sequences=2)

        for para in paraphrases:
            augmented_texts.append(para)
            augmented_labels.append(label)

    except Exception as e:
        print(f"Error for row {i}: {e}")
        continue

# Create new DataFrame for augmented data
aug_df = pd.DataFrame({
    'Text': augmented_texts,
    'Label': augmented_labels
})

# Merge original and augmented
df_augmented = pd.concat([df, aug_df], ignore_index=True)


 10%|▉         | 44/450 [04:45<30:52,  4.56s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 20%|█▉        | 89/450 [09:53<1:02:48, 10.44s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 30%|██▉       | 134/450 [14:22<38:34,  7.32s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 40%|███▉      | 179/450 [18:55<22:52,  5.07s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 50%|████▉     | 224/450 [25:52<32:59,  8.76s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

 70%|██████▉   | 314/450 [40:46<14:44,  6.50s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 80%|███████▉  | 359/450 [46:48<12:26,  8.20s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 90%|████████▉ | 404/450 [51:57<04:05,  5.35s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


100%|█████████▉| 449/450 [58:15<00:05,  5.37s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


100%|██████████| 450/450 [58:19<00:00,  7.78s/it]


In [16]:
print(df_augmented['Label'].value_counts())


Label
Anxiety                                            150
Depression                                         150
ADHD( Attention deficit hyperactivity disorder)    150
Bipolar Disorder                                   150
Autism Spectrum Disorder (ASD)                     150
Obsessive-Compulsive Disorder (OCD)                150
Schizophrenia                                      150
Post-Traumatic Stress Disorder (PTSD)              150
Eating Disorder                                    150
Name: count, dtype: int64


In [15]:
# Save your augmented dataset
df_augmented.to_csv("augmented_dataset.csv", index=False)


In [17]:
import os
os.listdir()


['.ipynb_checkpoints', 'augmented_dataset.csv', 'classifier_training.ipynb']

In [18]:

print(os.path.abspath("augmented_classification.csv"))


C:\Users\Asus\MindCheck_AI\notebooks\augmented_classification.csv
